links to ds_salaries datasets:

https://www.kaggle.com/datasets/murilozangari/jobs-and-salaries-in-data-field-2024

https://www.kaggle.com/datasets/sazidthe1/data-science-salaries

https://www.kaggle.com/datasets/lorenzovzquez/data-jobs-salaries

https://www.kaggle.com/datasets/arnabchaki/data-science-salaries-2023

In [ ]:
import pandas as pd
import pycountry
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('salaries.csv')
data1= pd.read_csv('jobs_in_data_2024.csv')
data2= pd.read_csv('ds_salaries.csv')
data3= pd.read_csv('data_science_salaries.csv')

In [ ]:
ds_salaries = pd.concat([data, data1, data2, data3], ignore_index=True)
ds_salaries

# Handling NaN values

In [ ]:
ds_salaries.isnull().sum()

In [ ]:
# in the concatenated ds.salaries we have following columns all referring to the working model: work_models, work_setting, remote_ratio
# so i'm going to combine them in one column 

In [ ]:
ds_salaries['remote_ratio'].value_counts(dropna=False)

In [ ]:
ds_salaries['work_models'].value_counts(dropna=False)

In [ ]:
ds_salaries['work_setting'].value_counts(dropna=False)

In [ ]:
#filling the NaN values in remote_ratio with the values from the other columns and then dropping the unnecessary columns
def fill_remote_ratio(row):
    if pd.isna(row['remote_ratio']):
        
        if row['work_models'] == 'Hybrid':
            return 50.0
        elif row['work_models'] == 'Remote':
            return 100.0
        elif row['work_models'] == 'On-site':
            return 0.0
      
        if pd.isna(row['work_models']):
            if row['work_setting'] == 'Hybrid':
                return 50.0
            elif row['work_setting'] == 'Remote':
                return 100.0
            elif row['work_setting'] == 'In-person':
                return 0
    return row['remote_ratio']

In [ ]:
ds_salaries['remote_ratio'] = ds_salaries.apply(fill_remote_ratio, axis=1)

In [ ]:
ds_salaries['remote_ratio'].value_counts(dropna=False)

In [ ]:
ds_salaries = ds_salaries.drop(columns=['work_models', 'work_setting', 'job_category'])

In [ ]:
ds_salaries.isnull().sum()

# Checking values in different columns

In [ ]:
#checking the values in experience_level and unifying them
ds_salaries['experience_level'].value_counts(dropna=False)

In [ ]:
level_mapping = {
    'SE': 'Senior',
    'Senior': 'Senior',
    'MI': 'Mid-level',
    'Mid-level': 'Mid-level',
    'Senior-level': 'Senior',
    'EN': 'Entry-level',
    'Entry-level': 'Entry-level',
    'EX': 'Executive',
    'Executive': 'Executive',
    'Executive-level': 'Executive'
}

ds_salaries['experience_level'] = ds_salaries['experience_level'].map(level_mapping)

In [ ]:
#checking the values in employment_type and unifying them
ds_salaries['employment_type'].value_counts(dropna=False)

In [ ]:
type_mapping = {
    'FT': 'Full-time',
    'Full-time': 'Full-time',
    'PT': 'Part-time',
    'Part-time': 'Part-time',
    'Contract': 'Contract',
    'CT': 'Contract',
    'FL': 'Freelance',
    'Freelance': 'Freelance'
}


ds_salaries['employment_type'] = ds_salaries['employment_type'].map(type_mapping)

In [ ]:
#checking the values in company_size and unifying them
ds_salaries['company_size'].value_counts(dropna=False)

In [ ]:
size_mapping = {
    'M': 'Medium',
    'Medium': 'Medium',
    'L': 'Large',
    'Large': 'Large',
    'S': 'Small',
    'Small': 'Small'
}


ds_salaries['company_size'] = ds_salaries['company_size'].map(size_mapping)


# Handling duplicates

In [ ]:
#checking for duplicates
ds_salaries.duplicated().sum()



In [ ]:
#dropping duplicates 
ds_salaries = ds_salaries.drop_duplicates()

In [ ]:
ds_salaries

In [ ]:
#changing country codes to names
country_dict = {country.alpha_2: country.name for country in pycountry.countries}

def code_to_name(code):
    return country_dict.get(code, code)


ds_salaries['company_location'] = ds_salaries['company_location'].apply(code_to_name)
ds_salaries['employee_residence'] = ds_salaries['employee_residence'].apply(code_to_name)

ds_salaries

In [ ]:
# changing all the salaries to EUR and dropping the unnecessary columns. Exchange rate 26.05.2024 @ 1$ = 0,92€ (0,92 is also close to average rate)

In [ ]:
# adding salaries in eur column @ 1usd = 0,92eur
ds_salaries['salary_in_eur'] = ds_salaries['salary_in_usd'] * 0.92

In [ ]:
#dropping unnecessary columns.
ds_salaries = ds_salaries.drop(columns=['salary', 'salary_in_usd', 'salary_currency'])
ds_salaries

In [ ]:
#listing countries to statistics
stats_countries = [
    'Austria', 'Belgium', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Finland', 'France',
    'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy', 'Liechtenstein', 'Luxembourg', 'Malta', 'Netherlands', 'Norway',
    'Poland', 'Portugal', 'Spain', 'Sweden',
    'Switzerland', 'United Kingdom', 'Vatican City', 'United States', 'Canada', 'Qatar', 'Japan', 'Australia', 'United Arab Emirates', 'Saudi Arabia']


In [ ]:
stats_countries= ds_salaries[ds_salaries['company_location'].isin(stats_countries)]

In [ ]:
#Grouping the job titles in categories 
job_titles = ['Data Manager', 'BI Developer', 'Data Analyst',
       'Machine Learning Engineer', 'Data Scientist', 'Data Engineer',
       'Applied Scientist', 'Data Architect', 'BI Analyst',
       'Head of Data', 'Analytics Engineer', 'Business Intelligence',
       'Data Quality Analyst', 'Research Engineer', 'Research Scientist',
       'AI Engineer', 'Business Intelligence Analyst',
       'Data Operations Manager', 'Machine Learning Scientist',
       'Business Intelligence Lead', 'Data Quality Engineer',
       'Data Integration Engineer', 'Cloud Database Engineer',
       'Data Developer', 'Research Analyst',
       'Data Visualization Developer', 'Data Integration Specialist',
       'Data Modeler', 'Robotics Engineer', 'Data Specialist',
       'AI Software Engineer', 'Business Intelligence Engineer',
       'AI Architect', 'Data Analytics Manager',
       'Data Analytics Specialist', 'Computer Vision Engineer',
       'Data Operations Associate', 'AI Developer',
       'Data Product Manager', 'Data Analytics Lead', 'Prompt Engineer',
       'Insight Analyst', 'Data Operations Analyst',
       'Data Reporting Analyst', 'Business Intelligence Developer',
       'Data Product Owner', 'MLOps Engineer',
       'Data Visualization Specialist', 'Lead AI Engineer',
       'AI Product Manager', 'Data Management Specialist',
       'Data Strategist', 'Robotics Software Engineer',
       'Machine Learning Researcher', 'Data Lead', 'AI Scientist',
       'Data Infrastructure Engineer', 'Machine Learning Developer',
       'Machine Learning Manager', 'Master Data Specialist',
       'Machine Learning Modeler', 'NLP Engineer',
       'Data Management Analyst',
       'Encounter Data Management Professional', 'Admin & Data Analyst',
       'Consultant Data Engineer', 'Business Intelligence Manager',
       'Data Quality Manager', 'Business Intelligence Specialist',
       'Director of Business Intelligence', 'AI Research Scientist',
       'Data Operations Specialist', 'Bear Robotics',
       'Lead Data Scientist', 'Applied Research Scientist',
       'Machine Learning Infrastructure Engineer', 'CRM Data Analyst',
       'ETL Developer', 'BI Data Analyst', 'Applied Data Scientist',
       'Computational Biologist', 'Data Analytics Associate',
       'Data Management Consultant', 'Data DevOps Engineer',
       'Big Data Developer', 'Quantitative Research Analyst',
       'Lead Machine Learning Engineer',
       'Machine Learning Research Engineer', 'Data Analytics Consultant',
       'AI Research Engineer', 'ETL Engineer', 'Head of Machine Learning',
       'Data Integration Developer', 'Data Pipeline Engineer',
       'Business Data Analyst', 'Marketing Data Scientist',
       'Deep Learning Engineer', 'Decision Scientist',
       'Financial Data Analyst', 'Data Strategy Manager',
       'Data Visualization Engineer', 'Principal Data Scientist',
       'Staff Data Analyst', 'Machine Learning Software Engineer',
       'AI Programmer', 'Applied Machine Learning Scientist',
       'Data Operations Engineer', 'Principal Data Engineer',
       'Power BI Developer', 'Staff Machine Learning Engineer',
       'Staff Data Scientist', 'Machine Learning Specialist',
       'Business Intelligence Data Analyst', 'Software Data Engineer',
       'Compliance Data Analyst', 'Cloud Data Engineer',
       'Analytics Engineering Manager', 'AWS Data Architect',
       'Product Data Analyst', 'Data Visualization Analyst',
       'Autonomous Vehicle Technician', 'Finance Data Analyst',
       'Applied Machine Learning Engineer', 'Big Data Engineer',
       'Lead Data Analyst', 'BI Data Engineer',
       'Deep Learning Researcher', 'Big Data Architect',
       'Computer Vision Software Engineer', 'Azure Data Engineer',
       'Manager Data Management', 'Principal Machine Learning Engineer',
       'Data Science Tech Lead', 'Data Scientist Lead',
       'Marketing Data Analyst', 'Data Analytics Engineer',
       'Cloud Data Architect', 'Lead Data Engineer',
       'Principal Data Analyst', 'Data Science', 'Data Science Manager',
       'Data Science Analyst', 'Data Science Consultant', 'ML Engineer',
       'Data Science Director', 'Data Science Engineer',
       'Machine Learning Operations Engineer',
       'Data Science Practitioner', 'ML Ops Engineer',
       'Data Science Lead', 'Director of Data Science',
       'Managing Director Data Science', 'Head of Data Science',
       'Data Modeller']


def categorize_job_title(job_title):
    job_title = job_title.lower()
    if 'engineer' in job_title or 'engineering' in job_title:
        return 'Data Engineering'
    elif 'science' in job_title or 'scientist' in job_title:
        return 'Data Science'
    elif 'analytics' in job_title or 'analyst' in job_title:
        return 'Data Analytics'
    elif 'BI' in job_title or 'Intillegence' in job_title:
        return 'Business Intellegence And Analytics'
    
    else:
        return 'Other'

job_categories = pd.DataFrame({'job_title': job_titles})

stats_countries['job_category'] = stats_countries['job_title'].apply(categorize_job_title)



In [ ]:
stats_countries = stats_countries.drop(columns=['job_title'])

In [ ]:
stat_countries = 'stats_countries.csv'
stats_countries.to_csv(stat_countries, index=False)

In [ ]:
ds_salaries['job_category'] = ds_salaries['job_title'].apply(categorize_job_title)

In [ ]:
ds_salaries = ds_salaries.drop(columns=['job_title'])

In [ ]:
datascience_salaries = 'ds_salaries.csv'
ds_salaries.to_csv(datascience_salaries, index=False)

In [ ]:
ds_salaries

# Building Machine Learning Model

In [ ]:
#Linearregression model

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
numdata = ds_salaries.select_dtypes(exclude='object')
catdata = ds_salaries.select_dtypes(include='object')

In [ ]:
transformer = MinMaxScaler().fit(numdata)
X_norm = transformer.transform(numdata)
print(numdata.shape)
X_numdata_scale = pd.DataFrame(X_norm, columns=numdata.columns)
X_numdata_scale.head()

In [ ]:
encoder = OneHotEncoder(drop='first').fit(catdata)  
encoded = encoder.transform(catdata).toarray() 
encoded_columns = OneHotEncoder(drop='first').fit(catdata).get_feature_names_out(input_features=catdata.columns) 

onehot_encoded_X = pd.DataFrame(encoded, columns = encoded_columns) 
onehot_encoded_X.head()

In [ ]:
concdata = pd.concat([onehot_encoded_X, X_numdata_scale ], axis=1)

In [ ]:
concdata

In [ ]:
X=concdata.drop(['salary_in_eur'], axis=1)
y=ds_salaries['salary_in_eur']

In [ ]:
from sklearn.model_selection import train_test_split as tts

X_train, X_test, y_train, y_test=tts(X, y, test_size=.2)

In [ ]:
from sklearn.linear_model import LinearRegression as LinReg

linreg=LinReg()    
linreg.fit(X_train, y_train)   
y_pred_linreg=linreg.predict(X_test) 

In [ ]:
print ('train R2: {} -- test R2: {}'.format(linreg.score(X_train, y_train),
                                            linreg.score(X_test, y_test)))

In [ ]:
from sklearn.metrics import mean_squared_error as mse


train_mse=mse(linreg.predict(X_train), y_train)
test_mse=mse(linreg.predict(X_test), y_test)

print ('train MSE: {} -- test MSE: {}'.format(train_mse, test_mse))

In [ ]:
print ('train RMSE: {} -- test RMSE: {}'.format(train_mse**.5, test_mse**.5))

In [ ]:
from sklearn.metrics import mean_absolute_error as mae

train_mae=mae(linreg.predict(X_train), y_train)
test_mae=mae(linreg.predict(X_test), y_test)

print ('train MAE: {} -- test MAE: {}'.format(train_mae, test_mae))

In [ ]:
#Randomforest Regressor

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error

In [ ]:
X = ds_salaries.drop('salary_in_eur', axis=1)  # Features
y = ds_salaries['salary_in_eur']  # Target variable


In [ ]:
X = pd.get_dummies(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

# Focusing on EU Market

In [ ]:
# focusing on european market, i'm going to only keep the data where companies are basaed in Europe. 

In [ ]:
#listing european countries
european_countries = [
    'Albania', 'Andorra', 'Austria', 'Belarus', 'Belgium', 'Bosnia and Herzegovina',
    'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 'Georgia',
    'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy', 'Kosovo', 'Latvia', 'Liechtenstein',
    'Lithuania', 'Luxembourg', 'Malta', 'Moldova', 'Monaco', 'Montenegro', 'Netherlands', 'North Macedonia', 'Norway',
    'Poland', 'Portugal', 'Romania', 'San Marino', 'Serbia', 'Slovakia', 'Slovenia', 'Spain', 'Sweden',
    'Switzerland', 'Ukraine', 'United Kingdom', 'Vatican City'
]


In [ ]:
# creating data science salaries for EU-countries (ds_eu) 
ds_eu= ds_salaries[ds_salaries['company_location'].isin(european_countries)]

In [ ]:
ds_eu

In [ ]:
europe_salaries = 'ds_eu.csv'
ds_eu.to_csv(europe_salaries, index=False)

# Descriptive Statistics

In [ ]:
# Summary statistics
summary_stats = ds_eu['salary_in_eur'].describe()
summary_stats

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Histogram
plt.figure(figsize=(10, 6))
sns.histplot(ds_eu['salary_in_eur'], kde=True)
plt.title('Salary Distribution')
plt.xlabel('Salary in EUR')
plt.ylabel('Frequency')
plt.show()


In [ ]:
# Average salary by experience level
avg_salary_exp = ds_eu.groupby('experience_level')['salary_in_eur'].mean()
print(avg_salary_exp)

# Average salary by employment type
avg_salary_emp_type = ds_eu.groupby('employment_type')['salary_in_eur'].mean()
print(avg_salary_emp_type)

# Average salary by company size
avg_salary_company_size = ds_eu.groupby('company_size')['salary_in_eur'].mean()
print(avg_salary_company_size)


In [ ]:
# Average salary by remote ratio
avg_salary_remote = ds_eu.groupby('remote_ratio')['salary_in_eur'].mean()
print(avg_salary_remote)


In [ ]:
# Average salary by employee residence
avg_salary_residence = ds_eu.groupby('employee_residence')['salary_in_eur'].mean().sort_values(ascending=False)
print(avg_salary_residence)

In [ ]:
# Bar Plot for average salary by employee residence
plt.figure(figsize=(10, 6))
avg_salary_residence.plot(kind='bar')
plt.title('Average Salary by Employee Residence')
plt.xlabel('Employee Residence')
plt.ylabel('Average Salary in EUR')
plt.show()



In [ ]:
# Average salary by company location
avg_salary_company_loc = ds_eu.groupby('company_location')['salary_in_eur'].mean().sort_values(ascending=False)
print(avg_salary_company_loc)

In [ ]:
# Bar Plot for average salary by company location
plt.figure(figsize=(10, 6))
avg_salary_company_loc.plot(kind='bar')
plt.title('Average Salary by Company Location')
plt.xlabel('Company Location')
plt.ylabel('Average Salary in EUR')
plt.show()

In [ ]:
# Salary by experience level
plt.figure(figsize=(10, 6))
sns.boxplot(x='experience_level', y='salary_in_eur', data=ds_eu)
plt.title('Salary by Experience Level')
plt.xlabel('Experience Level')
plt.ylabel('Salary in EUR')
plt.show()

# Predicting salaries in the EU-Market

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae


features = ['work_year', 'experience_level', 'employment_type', 'employee_residence', 'remote_ratio', 'company_location', 'company_size', 'job_category']
df_encoded = pd.get_dummies(ds_eu[features], drop_first=True)
X = df_encoded
y = ds_eu['salary_in_eur']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model = LinearRegression()
model.fit(X_train, y_train)


y_pred = model.predict(X_test)


mae = mae(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')
print(f'R2 Score: {r2}')


In [ ]:
print ('train R2: {} -- test R2: {}'.format(model.score(X_train, y_train),
                                            model.score(X_test, y_test)))

In [ ]:
train_mse =mse(model.predict(X_train), y_train)
test_mse =mse(model.predict(X_test), y_test)

print ('train MSE: {} -- test MSE: {}'.format(train_mse, test_mse))

In [ ]:
print ('train RMSE: {} -- test RMSE: {}'.format(train_mse**.5, test_mse**.5))

In [ ]:
from sklearn.metrics import mean_absolute_error as mae
train_mae= mae(model.predict(X_train), y_train)
test_mae= mae(model.predict(X_test), y_test)

print ('train MAE: {} -- test MAE: {}'.format(train_mae, test_mae))

# Webscraping Cost Of Living From Numbeo.com

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url= 'https://www.numbeo.com/cost-of-living/rankings_by_country.jsp?title=2024&region=150'

In [ ]:
response = requests.get(url)

In [ ]:
response.status_code

In [ ]:
soup = BeautifulSoup(response.content, "html.parser")

In [ ]:
soup.select("#t2")

In [ ]:
html = '''[<table class="stripe row-border order-column compact" id="t2">
 <thead>
 <tr>
 <th><div style="font-size: 80%; vertical-align: middle;">Rank</div></th>
 <th>Country</th>
 <th><div style="font-size: 90%;">Cost of Living Index</div></th>
 <th><div style="font-size: 90%;">Rent Index</div></th>
 <th><div style="font-size: 90%;">Cost of Living Plus Rent Index</div></th>
 <th><div style="font-size: 90%;">Groceries Index</div></th>
 <th><div style="font-size: 90%;">Restaurant Price Index</div></th>
 <th><div style="font-size: 90%;">Local Purchasing Power Index</div></th>
 </tr>
 </thead>
 <tbody>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Switzerland</td>
 <td style="text-align: right">112.2</td>
 <td style="text-align: right">52.8</td>
 <td style="text-align: right">84.0</td>
 <td style="text-align: right">116.7</td>
 <td style="text-align: right">104.1</td>
 <td style="text-align: right">118.9</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Iceland</td>
 <td style="text-align: right">85.7</td>
 <td style="text-align: right">41.0</td>
 <td style="text-align: right">64.5</td>
 <td style="text-align: right">86.5</td>
 <td style="text-align: right">88.0</td>
 <td style="text-align: right">109.1</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Norway</td>
 <td style="text-align: right">79.2</td>
 <td style="text-align: right">27.8</td>
 <td style="text-align: right">54.8</td>
 <td style="text-align: right">77.5</td>
 <td style="text-align: right">75.9</td>
 <td style="text-align: right">94.9</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Denmark</td>
 <td style="text-align: right">75.2</td>
 <td style="text-align: right">28.6</td>
 <td style="text-align: right">53.1</td>
 <td style="text-align: right">65.0</td>
 <td style="text-align: right">82.6</td>
 <td style="text-align: right">103.3</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Jersey</td>
 <td style="text-align: right">74.3</td>
 <td style="text-align: right">52.5</td>
 <td style="text-align: right">64.0</td>
 <td style="text-align: right">64.1</td>
 <td style="text-align: right">93.6</td>
 <td style="text-align: right">72.0</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Guernsey</td>
 <td style="text-align: right">67.5</td>
 <td style="text-align: right">46.7</td>
 <td style="text-align: right">57.6</td>
 <td style="text-align: right">68.8</td>
 <td style="text-align: right">63.6</td>
 <td style="text-align: right">83.2</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Austria</td>
 <td style="text-align: right">66.8</td>
 <td style="text-align: right">23.7</td>
 <td style="text-align: right">46.4</td>
 <td style="text-align: right">64.8</td>
 <td style="text-align: right">59.0</td>
 <td style="text-align: right">84.4</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Ireland</td>
 <td style="text-align: right">66.5</td>
 <td style="text-align: right">47.3</td>
 <td style="text-align: right">57.4</td>
 <td style="text-align: right">58.9</td>
 <td style="text-align: right">65.9</td>
 <td style="text-align: right">82.0</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">France</td>
 <td style="text-align: right">66.0</td>
 <td style="text-align: right">22.3</td>
 <td style="text-align: right">45.3</td>
 <td style="text-align: right">68.6</td>
 <td style="text-align: right">57.9</td>
 <td style="text-align: right">83.4</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Finland</td>
 <td style="text-align: right">65.5</td>
 <td style="text-align: right">20.8</td>
 <td style="text-align: right">44.3</td>
 <td style="text-align: right">61.6</td>
 <td style="text-align: right">63.8</td>
 <td style="text-align: right">97.3</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Luxembourg</td>
 <td style="text-align: right">65.3</td>
 <td style="text-align: right">42.4</td>
 <td style="text-align: right">54.4</td>
 <td style="text-align: right">65.1</td>
 <td style="text-align: right">69.4</td>
 <td style="text-align: right">148.9</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Netherlands</td>
 <td style="text-align: right">64.3</td>
 <td style="text-align: right">34.5</td>
 <td style="text-align: right">50.2</td>
 <td style="text-align: right">57.4</td>
 <td style="text-align: right">61.0</td>
 <td style="text-align: right">103.3</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">United Kingdom</td>
 <td style="text-align: right">63.7</td>
 <td style="text-align: right">33.5</td>
 <td style="text-align: right">49.4</td>
 <td style="text-align: right">55.5</td>
 <td style="text-align: right">63.9</td>
 <td style="text-align: right">90.1</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Belgium</td>
 <td style="text-align: right">63.0</td>
 <td style="text-align: right">21.4</td>
 <td style="text-align: right">43.3</td>
 <td style="text-align: right">56.1</td>
 <td style="text-align: right">67.8</td>
 <td style="text-align: right">90.7</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Germany</td>
 <td style="text-align: right">62.7</td>
 <td style="text-align: right">26.3</td>
 <td style="text-align: right">45.4</td>
 <td style="text-align: right">57.6</td>
 <td style="text-align: right">52.5</td>
 <td style="text-align: right">101.0</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Sweden</td>
 <td style="text-align: right">62.2</td>
 <td style="text-align: right">23.1</td>
 <td style="text-align: right">43.6</td>
 <td style="text-align: right">61.8</td>
 <td style="text-align: right">56.1</td>
 <td style="text-align: right">94.6</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Isle Of Man</td>
 <td style="text-align: right">59.8</td>
 <td style="text-align: right">27.4</td>
 <td style="text-align: right">44.5</td>
 <td style="text-align: right">48.7</td>
 <td style="text-align: right">67.9</td>
 <td style="text-align: right">167.1</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Italy</td>
 <td style="text-align: right">58.9</td>
 <td style="text-align: right">21.1</td>
 <td style="text-align: right">41.0</td>
 <td style="text-align: right">56.8</td>
 <td style="text-align: right">55.0</td>
 <td style="text-align: right">62.8</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Malta</td>
 <td style="text-align: right">55.1</td>
 <td style="text-align: right">25.3</td>
 <td style="text-align: right">40.9</td>
 <td style="text-align: right">49.9</td>
 <td style="text-align: right">57.5</td>
 <td style="text-align: right">58.3</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Greece</td>
 <td style="text-align: right">54.2</td>
 <td style="text-align: right">13.6</td>
 <td style="text-align: right">34.9</td>
 <td style="text-align: right">47.9</td>
 <td style="text-align: right">52.9</td>
 <td style="text-align: right">42.1</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Estonia</td>
 <td style="text-align: right">53.0</td>
 <td style="text-align: right">15.5</td>
 <td style="text-align: right">35.2</td>
 <td style="text-align: right">45.2</td>
 <td style="text-align: right">48.8</td>
 <td style="text-align: right">69.0</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Latvia</td>
 <td style="text-align: right">51.4</td>
 <td style="text-align: right">11.7</td>
 <td style="text-align: right">32.6</td>
 <td style="text-align: right">42.6</td>
 <td style="text-align: right">46.2</td>
 <td style="text-align: right">51.9</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Slovenia</td>
 <td style="text-align: right">49.3</td>
 <td style="text-align: right">17.0</td>
 <td style="text-align: right">34.0</td>
 <td style="text-align: right">46.3</td>
 <td style="text-align: right">41.2</td>
 <td style="text-align: right">68.0</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Czech Republic</td>
 <td style="text-align: right">48.6</td>
 <td style="text-align: right">21.0</td>
 <td style="text-align: right">35.5</td>
 <td style="text-align: right">44.1</td>
 <td style="text-align: right">34.5</td>
 <td style="text-align: right">64.9</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Lithuania</td>
 <td style="text-align: right">48.4</td>
 <td style="text-align: right">15.8</td>
 <td style="text-align: right">32.9</td>
 <td style="text-align: right">43.6</td>
 <td style="text-align: right">45.1</td>
 <td style="text-align: right">61.8</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Spain</td>
 <td style="text-align: right">48.4</td>
 <td style="text-align: right">23.6</td>
 <td style="text-align: right">36.6</td>
 <td style="text-align: right">45.1</td>
 <td style="text-align: right">44.0</td>
 <td style="text-align: right">78.2</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Slovakia</td>
 <td style="text-align: right">47.9</td>
 <td style="text-align: right">15.1</td>
 <td style="text-align: right">32.4</td>
 <td style="text-align: right">45.1</td>
 <td style="text-align: right">35.5</td>
 <td style="text-align: right">55.8</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Croatia</td>
 <td style="text-align: right">46.9</td>
 <td style="text-align: right">14.5</td>
 <td style="text-align: right">31.5</td>
 <td style="text-align: right">42.6</td>
 <td style="text-align: right">42.6</td>
 <td style="text-align: right">58.0</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Portugal</td>
 <td style="text-align: right">46.6</td>
 <td style="text-align: right">25.4</td>
 <td style="text-align: right">36.5</td>
 <td style="text-align: right">42.9</td>
 <td style="text-align: right">37.9</td>
 <td style="text-align: right">46.5</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Hungary</td>
 <td style="text-align: right">43.4</td>
 <td style="text-align: right">11.3</td>
 <td style="text-align: right">28.2</td>
 <td style="text-align: right">39.5</td>
 <td style="text-align: right">36.7</td>
 <td style="text-align: right">49.8</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Poland</td>
 <td style="text-align: right">41.9</td>
 <td style="text-align: right">19.2</td>
 <td style="text-align: right">31.1</td>
 <td style="text-align: right">35.9</td>
 <td style="text-align: right">35.7</td>
 <td style="text-align: right">66.2</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Montenegro</td>
 <td style="text-align: right">40.9</td>
 <td style="text-align: right">16.6</td>
 <td style="text-align: right">29.4</td>
 <td style="text-align: right">36.3</td>
 <td style="text-align: right">35.6</td>
 <td style="text-align: right">41.8</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Albania</td>
 <td style="text-align: right">40.8</td>
 <td style="text-align: right">10.2</td>
 <td style="text-align: right">26.3</td>
 <td style="text-align: right">39.4</td>
 <td style="text-align: right">31.0</td>
 <td style="text-align: right">31.6</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Romania</td>
 <td style="text-align: right">38.8</td>
 <td style="text-align: right">10.6</td>
 <td style="text-align: right">25.4</td>
 <td style="text-align: right">36.4</td>
 <td style="text-align: right">35.0</td>
 <td style="text-align: right">48.4</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Bulgaria</td>
 <td style="text-align: right">38.7</td>
 <td style="text-align: right">9.9</td>
 <td style="text-align: right">25.1</td>
 <td style="text-align: right">37.6</td>
 <td style="text-align: right">32.8</td>
 <td style="text-align: right">52.6</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Serbia</td>
 <td style="text-align: right">38.3</td>
 <td style="text-align: right">12.7</td>
 <td style="text-align: right">26.2</td>
 <td style="text-align: right">35.0</td>
 <td style="text-align: right">30.4</td>
 <td style="text-align: right">39.6</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Bosnia And Herzegovina</td>
 <td style="text-align: right">34.7</td>
 <td style="text-align: right">5.7</td>
 <td style="text-align: right">21.0</td>
 <td style="text-align: right">32.7</td>
 <td style="text-align: right">22.0</td>
 <td style="text-align: right">47.1</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Moldova</td>
 <td style="text-align: right">34.6</td>
 <td style="text-align: right">9.8</td>
 <td style="text-align: right">22.8</td>
 <td style="text-align: right">31.0</td>
 <td style="text-align: right">26.6</td>
 <td style="text-align: right">35.2</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">North Macedonia</td>
 <td style="text-align: right">34.1</td>
 <td style="text-align: right">6.8</td>
 <td style="text-align: right">21.1</td>
 <td style="text-align: right">31.2</td>
 <td style="text-align: right">22.8</td>
 <td style="text-align: right">37.0</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Kosovo (Disputed Territory)</td>
 <td style="text-align: right">31.3</td>
 <td style="text-align: right">8.5</td>
 <td style="text-align: right">20.5</td>
 <td style="text-align: right">29.3</td>
 <td style="text-align: right">20.7</td>
 <td style="text-align: right">34.8</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Russia</td>
 <td style="text-align: right">29.5</td>
 <td style="text-align: right">9.9</td>
 <td style="text-align: right">20.2</td>
 <td style="text-align: right">25.9</td>
 <td style="text-align: right">26.2</td>
 <td style="text-align: right">41.5</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Ukraine</td>
 <td style="text-align: right">28.6</td>
 <td style="text-align: right">8.5</td>
 <td style="text-align: right">19.0</td>
 <td style="text-align: right">26.6</td>
 <td style="text-align: right">22.2</td>
 <td style="text-align: right">32.2</td>
 </tr>
 <tr style="width: 100%">
 <td></td>
 <td class="cityOrCountryInIndicesTable">Belarus</td>
 <td style="text-align: right">27.1</td>
 <td style="text-align: right">8.2</td>
 <td style="text-align: right">18.2</td>
 <td style="text-align: right">24.4</td>
 <td style="text-align: right">25.3</td>
 <td style="text-align: right">39.6</td>
 </tr>
 </tbody>
 </table>]'''

In [ ]:
soup = BeautifulSoup(html, 'html.parser')


headers = []
for th in soup.find_all('thead')[0].find_all('th'):
    if th.div:
        headers.append(th.div.text.strip())
    else:
        headers.append(th.text.strip())

rows = []
for tr in soup.find_all('tbody')[0].find_all('tr'):
    row = []
    for td in tr.find_all('td'):
        row.append(td.text.strip())
    rows.append(row)


cost_of_living = pd.DataFrame(rows, columns=headers)


In [ ]:
ds_eu_countries= ds_eu['company_location'].unique()

In [ ]:
ds_eu['company_location'].unique()

In [ ]:
cost_of_living= cost_of_living[cost_of_living['Country'].isin(ds_eu_countries)]

In [ ]:
cost_of_living.reset_index(drop=True, inplace=True)

In [ ]:
cost_countries= cost_of_living['Country'].unique()

In [ ]:
ds_eu= ds_eu[ds_eu['company_location'].isin(cost_countries)]

In [ ]:
avg_salaries = ds_eu.groupby('company_location')['salary_in_eur'].mean().reset_index()

In [ ]:
avg_salaries.columns = ['country', 'avg_salary']

In [ ]:
cost_of_living.columns = cost_of_living.columns.str.lower().str.replace(' ', '_')

In [ ]:
costs_sals = pd.merge(cost_of_living, avg_salaries, on='country', how='inner')

In [ ]:
costs_sals.drop(columns=['rank'], inplace=True)

In [ ]:
costs_sals.drop(columns=['rent_index', 'cost_of_living_plus_rent_index', 'groceries_index', 'restaurant_price_index', 'local_purchasing_power_index'], inplace=True)

In [ ]:
costs_sals['cost_of_living_index'] = costs_sals['cost_of_living_index'].astype(float)

In [ ]:
costs_sals['salary_index'] = (costs_sals['avg_salary'] / costs_sals['cost_of_living_index']) * 100


In [ ]:
costs_sals

In [ ]:
col = 'cost_of_living.csv'
cost_of_living.to_csv(col, index=False)

In [ ]:
costs_salaries = 'cost_sals.csv'
costs_sals.to_csv(costs_salaries, index=False)

In [ ]:
eu_int_countries = ['Germany', 'Netherlands', 'Spain','France', 'Hungary', 'Austria', 'Switzerland', 'United Kingdom', 'Belgium']

In [ ]:
cost_sal_index = costs_sals[costs_sals['country'].isin(eu_int_countries)]

In [ ]:
cost_sal_ind= 'cost_sal_index.csv'
cost_sal_index.to_csv(cost_sal_ind, index=False)

In [ ]:
int_countries = ds_eu[ds_eu['company_location'].isin(eu_int_countries)]

In [ ]:
int_coun= 'int_countries.csv'
int_countries.to_csv(int_coun, index=False)

In [ ]:
int_countries.columns